In [8]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pylab as pl

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score

from pylab import rcParams
rcParams['figure.figsize'] = 12, 8
sns.set(font_scale = 1.7)

1. preprosessing

In [9]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from sklearn.metrics import mean_squared_error
from math import sqrt
%matplotlib inline
pd.options.display.max_columns = None

import seaborn as sns
pd.set_option('max_columns',200)
pd.set_option('max_rows',200)
from sklearn.ensemble import RandomForestRegressor
#记住这里是在做regression，所以import的是randomforestregressor


In [10]:
data = pd.read_csv('wholedata.csv')
print(data.shape)
data.head()

(18249, 70)


,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,month,winter,spring,summer,autumn,region_Albany,region_Atlanta,region_BaltimoreWashington,region_Boise,region_Boston,region_BuffaloRochester,region_California,region_Charlotte,region_Chicago,region_CincinnatiDayton,region_Columbus,region_DallasFtWorth,region_Denver,region_Detroit,region_GrandRapids,region_GreatLakes,region_HarrisburgScranton,region_HartfordSpringfield,region_Houston,region_Indianapolis,region_Jacksonville,region_LasVegas,region_LosAngeles,region_Louisville,region_MiamiFtLauderdale,region_Midsouth,region_Nashville,region_NewOrleansMobile,region_NewYork,region_Northeast,region_NorthernNewEngland,region_Orlando,region_Philadelphia,region_PhoenixTucson,region_Pittsburgh,region_Plains,region_Portland,region_RaleighGreensboro,region_RichmondNorfolk,region_Roanoke,region_Sacramento,region_SanDiego,region_SanFrancisco,region_Seattle,region_SouthCarolina,region_SouthCentral,region_Southeast,region_Spokane,region_StLouis,region_Syracuse,region_Tampa,region_TotalUS,region_West,region_WestTexNewMexico
0,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,0,2015,12,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,0,2015,12,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,0,2015,12,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,0,2015,12,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,0,2015,11,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
X = data.drop('AveragePrice',axis=1)
feature_list = list(X.columns)
print(len(feature_list))

y = data['AveragePrice']

X = X.values
y = y.values
print(X.shape,y.shape)

69
(18249, 69) (18249,)


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)
print(X_train.shape, y_train.shape)
print('---------------------')
print(X_test.shape, y_test.shape)

(14599, 69) (14599,)
---------------------
(3650, 69) (3650,)


In [13]:
def RMSE(y_true, y_pred):
    return sqrt(mean_squared_error(y_true,y_pred))

In [14]:
rf = RandomForestRegressor(random_state = 2020)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2020,
 'verbose': 0,
 'warm_start': False}


In [19]:
rf.fit(X_train, y_train)
prediction_test = rf.predict(X_test)
prediction_train = rf.predict(X_train)  
print('train: ', RMSE(y_train, prediction_train))
print('test: ', RMSE(y_test, prediction_test))

train:  0.0471646560080167
test:  0.12743018584995247


In [24]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
KFold(n_splits=5, random_state=2020, shuffle=False)
rmse_all = np.zeros(5)
i=0
for train_index, test_index in kf.split(X_train):
    #建模
    rf = RandomForestRegressor(random_state=2020)
    #训练
    rf.fit(X_train[train_index],y_train[train_index])
    #prediction
    prediction = rf.predict(X_train[test_index]) 
    prediction_train = rf.predict(X_train[train_index])  
    #rmse
    rmse = RMSE(y_train[test_index], prediction)
    print('-'*20,i,'-'*20)
    print('rmse: ',rmse)
    rmse_all[i]=rmse
    i += 1 
print('rmse_mean: ', np.mean(rmse_all))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


-------------------- 0 --------------------
rmse:  0.12761936440537885
-------------------- 1 --------------------
rmse:  0.1315594931049348
-------------------- 2 --------------------
rmse:  0.13385685617556345
-------------------- 3 --------------------
rmse:  0.12889424113303227
-------------------- 4 --------------------
rmse:  0.1339039809659658
rmse_mean:  0.13116678715697502


In [0]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(55, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [0]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 2020)
# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 10, scoring='neg_root_mean_squared_error', 
                              cv = 5, verbose=3, random_state=2020, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, y_train);

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 18.8min finished


In [0]:
rf_random.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600}

In [0]:
best_random = rf_random.best_estimator_

best_random.fit(X_train, y_train)

prediction_test = best_random.predict(X_test)

RMSE = rmse_test = sqrt(mean_squared_error(y_test, prediction_test))

print('rmse - test: ',rmse_test)

rmse - test:  0.1157557482505474


In [0]:
best_random = rf_random.best_estimator_

best_random.fit(X_train, y_train)

prediction_train = best_random.predict(X_train)

RMSE = rmse_train = sqrt(mean_squared_error(y_train, prediction_train))

print('rmse - train: ',rmse_train)

rmse - train:  1.2298399700448407e-14


In [0]:
from sklearn.metrics import mean_absolute_error
# predict the result for test
mae_test=mean_absolute_error(y_test,prediction_test)
print('mae: ',mae_test)

mae:  0.08124950228310493


In [0]:
rf.fit(X_train, y_train)

prediction_test = rf.predict(X_test)

RMSE = rmse_test = sqrt(mean_squared_error(y_test, prediction_test))

print('rmse - test: ',rmse_test)

RMSE = rmse_train = sqrt(mean_squared_error(y_train, prediction_train))
print('rmse - train: ',rmse_train)

rmse - test:  0.12743018584995247
rmse - train:  1.2298399700448407e-14


In [0]:
# get the feature importance
importances = list(rf.feature_importances_)

# transform the formula
feature_importances = [(feature, importance) for feature, importance in zip(data_feature_list, importances)]

# descending order
feature_importances = sorted(feature_importances, key= lambda x:x[1], reverse=True)

# print out
for a in feature_importances:
    print('Variable: {}      Importance:{}'.format(a[0],a[1]))

NameError: ignored